## Imports necessários

In [84]:
from pymongo import MongoClient
import pymongoarrow
from pymongoarrow.api import Schema
import pandas
from pymongoarrow.monkey import patch_all
from datetime import datetime
import pyarrow as pa
patch_all()

## Conexão com o mongodb

In [ ]:
client = MongoClient("mongodb://mongo1:27017,mongo2:27017,mongo3:27017/loja?replicaSet=db-replica-set&readPreference=secondaryPreferred&readPreferenceTags=dc:SP")

In [72]:
client.list_database_names()

['admin', 'config', 'local', 'loja', 'loja;']

### Configuração do banco de dados e collection

In [73]:
client = client["loja"]
colecao = db["produtos"]


## Convertendo para o Pandas os documentos do Mongodb

In [74]:
df= colecao.find_pandas_all({"nomeProduto": "Produto 19"})

In [75]:
print(df.head())

                        _id  idProduto nomeProduto  valorProduto  \
0  67b29c3b3c6d96f500686596       3478  Produto 19       9508.52   

                                idSkus  \
0  [87584, 56850, 16622, 39659, 85695]   

                                                skus  \
0  [{'idSku': 87584, 'nome': 'Produto 19-0', 'val...   

                                 marcas  \
0  [{'idMarca': 8, 'nome': 'Marca 47'}]   

                                    categorias  \
0  [{'idCategoria': 3, 'nome': 'Categoria 6'}]   

                                             imagens  
0  [{'idImagem': 5445, 'url': 'https://example.co...  


## Convertendo para o Apache Arrow os documentos do Mongodb

In [127]:
arrow_table = colecao.find_arrow_all({"nomeProduto": "Produto 19"})

In [128]:
print(arrow_table)

pyarrow.Table
_id: extension<pymongoarrow.objectid<ObjectIdType>>
idProduto: int32
nomeProduto: string
valorProduto: double
idSkus: list<item: int32>
  child 0, item: int32
skus: list<item: struct<idSku: int32, nome: string, valor: double, estoque: int32, cor: string>>
  child 0, item: struct<idSku: int32, nome: string, valor: double, estoque: int32, cor: string>
      child 0, idSku: int32
      child 1, nome: string
      child 2, valor: double
      child 3, estoque: int32
      child 4, cor: string
marcas: list<item: struct<idMarca: int32, nome: string>>
  child 0, item: struct<idMarca: int32, nome: string>
      child 0, idMarca: int32
      child 1, nome: string
categorias: list<item: struct<idCategoria: int32, nome: string>>
  child 0, item: struct<idCategoria: int32, nome: string>
      child 0, idCategoria: int32
      child 1, nome: string
imagens: list<item: struct<idImagem: int32, url: string>>
  child 0, item: struct<idImagem: int32, url: string>
      child 0, idImagem: i

## Configurando o schema para o resultado

In [116]:
# Definir um Schema para os produtos
produto_schema = Schema({
    "idProduto": pa.int32(),
    "nomeProduto": pa.string(),
    "valorProduto": pa.float64(),
    "idSkus": pa.list_(pa.int32()),  # Array de inteiros
    "skus": pa.list_(pa.struct({     # Array de JSONs embutidos
        "idSku": pa.int32(),
        "nome": pa.string(),
        "valor": pa.float64()
    })),
     "categorias": pa.list_(pa.struct({     # Array de JSONs embutidos
        "idCategoria": pa.int32(),
        "nome": pa.string()     
    })),  # Array de strings
     "marcas": pa.list_(pa.struct({     # Array de JSONs embutidos
        "idMarca": pa.int32(),
        "nome": pa.string()     
    })),  # Array de strings
    "imagens": pa.list_(pa.struct({   # Array de JSONs para imagens
        "url": pa.string()      
    }))
})


In [117]:
df= colecao.find_pandas_all({"nomeProduto": "Produto 19"},schema=produto_schema)

In [118]:
# Exibir a estrutura do schema retornado
print(df)

   idProduto nomeProduto  valorProduto                               idSkus  \
0       3478  Produto 19       9508.52  [87584, 56850, 16622, 39659, 85695]   

                                                skus  \
0  [{'idSku': 87584, 'nome': 'Produto 19-0', 'val...   

                                    categorias  \
0  [{'idCategoria': 3, 'nome': 'Categoria 6'}]   

                                 marcas  \
0  [{'idMarca': 8, 'nome': 'Marca 47'}]   

                                      imagens  
0  [{'url': 'https://example.com/img63.jpg'}]  


## Consultando as informações com agregações

In [142]:
df = colecao.aggregate_pandas_all([{ "$unwind": "$skus" },{ "$project": {"_id":0, "nomeProduto": 1, "idSku": "$skus.idSku", "nomeSku": "$skus.nome", "valor": "$skus.valor" } }])

In [143]:
print(df)

      nomeProduto  idSku        nomeSku    valor
0       Produto 0  63028    Produto 0-0  2723.98
1       Produto 0  19700    Produto 0-1  3264.95
2       Produto 0  25938    Produto 0-2  4079.07
3       Produto 1  99984    Produto 1-0  2818.89
4       Produto 1  80577    Produto 1-1  3192.57
...           ...    ...            ...      ...
1513  Produto 498  95710  Produto 498-2  1004.77
1514  Produto 499  98680  Produto 499-0  1245.49
1515  Produto 499  43905  Produto 499-1   548.67
1516  Produto 499  50365  Produto 499-2  2415.11
1517  Produto 499  43861  Produto 499-3  4135.01

[1518 rows x 4 columns]


## Escrevendo o resultado em uma coleção do Mongodb

In [144]:
from pymongoarrow.api import write
colecao_sku = db["skus"]
write(colecao_sku, df)

{'insertedCount': 1518}